In [4]:
import tensorflow as tf
print(tf.__version__)

2.18.0


In [5]:
import torch
import torch.nn as nn

In [6]:
#GELU Activation function
class GELU(nn.Module):
    def __init__(self,):
        super().__init__()
    def forward(self, x):
        return 0.5*x*(1+torch.tanh(torch.sqrt(torch.tensor(2.0/torch.pi))*(x+0.0447*torch.pow(x,3))))

In [7]:
#Feed Forward Neural network
class FeedForward(nn.Module):
    def __init__(self, cfg):
        super().__init__()
        self.layers = nn.Sequential(nn.Linear(cfg['emb_dim'], 4*cfg['emb_dim']), #incresing dimesion
                                    GELU(),
                                    nn.Linear(4*cfg['emb_dim'],cfg['emb_dim']))  #coming back to original dimension
    def forward(self,x):
        return self.layers(x)

In [8]:
# Layer Normalization
class LayerNorm(nn.Module):
    def __init__(self, emd_dim):
        super().__init__()
        self.eps = 1e-5
        self.scale = nn.Parameter(torch.ones(emd_dim))
        self.shift = nn.Parameter(torch.zeros(emd_dim))
    def forward(self, x):
        mean = x.mean(dim = -1, keepdim =True)
        var = x.var(dim =-1, keepdim = True, unbiased= False)
        norm_x = (x-mean)/(torch.sqrt(var+self.eps))
        return self.scale*norm_x + self.shift

In [9]:
class MultiHeadAttention(nn.Module):
    def __init__(self,d_in,d_out,context_length,dropout,num_heads,qkv_bias = False):
        super().__init__()
        assert(d_out % num_heads==0),\
            "d_out must be divisible by nums head"
        
        self.d_out = d_out
        self.num_head = num_heads
        self.head_dim = d_out//num_heads
        # self.d_in = d_in
        self.w_query = nn.Linear(d_in,d_out, bias=qkv_bias)
        self.w_key = nn.Linear(d_in,d_out, bias=qkv_bias)
        self.w_value = nn.Linear(d_in,d_out, bias=qkv_bias)

        self.out_proj = nn.Linear(d_out, d_out)  # Linear layer to combine head outputs
        self.dropout = nn.Dropout(dropout)
        self.register_buffer('mask',torch.triu(torch.ones(context_length,context_length),diagonal=1))

    def forward(self,x):
        b,num_token,d_in = x.shape

        keys = self.w_key(x) # Shape: (b, num_tokens, d_out)
        queries = self.w_query(x)
        values = self.w_value(x)

        keys = keys.view(b,num_token,self.num_head,self.head_dim)
        queries = queries.view(b,num_token,self.num_head,self.head_dim)
        values = values.view(b,num_token,self.num_head,self.head_dim)

        #grouping by num_heads
        keys = keys.transpose(1,2)
        values = values.transpose(1,2)
        queries = queries.transpose(1,2)

        # calculating attention score
        attn_score = queries @ keys.transpose(2,3)

        # calculating attention weigths,masking, scaling and dropout
        mask_bool = self.mask.bool()[:num_token,:num_token]
        attn_score= attn_score.masked_fill_(mask_bool, - torch.inf)
        attn_weight = torch.softmax(attn_score/keys.shape[-1]**0.5, dim=-1)
        attn_weight = self.dropout(attn_weight)
        
        #calculating the context vector
        context_vector = attn_weight @ values #ntokn x ntoken * ntoken x head_dim
        # trasposing to get all the context vextor togeth
        context_vector = context_vector.transpose(1,2)

        # combining heads 
        context_vector = context_vector.contiguous().view(b,num_token,self.d_out)
        context_vector = self.out_proj(context_vector) # optional projection
        return context_vector

In [10]:
class TransformerBlock(nn.Module):
    def __init__(self,cfg):
        super().__init__()
        self.attn = MultiHeadAttention(d_in=cfg['emb_dim'], d_out= cfg['emb_dim'],
                                       context_length=cfg['context_length'],
                                       num_heads=cfg['n_heads'], dropout= cfg['drop_rate'],
                                       qkv_bias= cfg['qkv_bias'])
        self.ff = FeedForward(cfg=cfg)
        self.norm1 = LayerNorm(emd_dim=cfg['emb_dim'])
        self.norm2 = LayerNorm(emd_dim=cfg['emb_dim'])

        self.drop_shortcut  = nn.Dropout(cfg['drop_rate'])
    
    def forward(self,x):
        shortcut = x
        x= self.norm1(x)
        x = self.attn(x)
        x = self.drop_shortcut(x)

        x = x+ shortcut

        shortcut =x
        x = self.norm2(x)
        x = self.ff(x)
        x = self.drop_shortcut(x)
        x = x + shortcut

        return x


In [11]:
class GPTModel(nn.Module):
    def __init__(self,cfg):
        super().__init__()
        self.tok_emb = nn.Embedding(cfg['vocab_size'], cfg['emb_dim'])
        self.pos_emb = nn.Embedding(cfg['context_length'], cfg['emb_dim'])
        self.drop_emb = nn.Dropout(cfg['drop_rate'])
        self.trf_blocks = nn.Sequential(*[TransformerBlock(cfg) for _ in range(cfg['n_layers'])])
        self.final_norm  = LayerNorm(cfg['emb_dim'])
        self.out_head = nn.Linear(cfg['emb_dim'],cfg['vocab_size'], bias=False)
    def forward(self,in_idx):
        batch_size, seq_len = in_idx.shape
        token_embded = self.tok_emb(in_idx)
        pos_embded = self.pos_emb(torch.arange(seq_len, device=in_idx.device))
        
        x = token_embded+pos_embded
        x = self.drop_emb(x)
        x = self.trf_blocks(x)
        x = self.final_norm(x)
        logits = self.out_head(x)
        return logits



In [12]:
# gpt-2 configurations
GPT_CONFIG_124M= {
    'vocab_size' :50257,
    'context_length': 256,
    'emb_dim':768,
    'n_heads':12,
    'n_layers':12,
    'drop_rate':0.1,
    'qkv_bias':False
}

In [13]:
from zz3_weightDownload import download_and_load_gpt2

In [14]:
from zz3_weightDownload import download_and_load_gpt2

In [15]:
settings, params = download_and_load_gpt2(model_size="124M", models_dir="gpt_2")

c:\Users\hp\anaconda3\Lib\site-packages\urllib3\connectionpool.py:1100: InsecureRequestWarning: Unverified HTTPS request is being made to host 'openaipublic.blob.core.windows.net'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


File already exists and is up to date


c:\Users\hp\anaconda3\Lib\site-packages\urllib3\connectionpool.py:1100: InsecureRequestWarning: Unverified HTTPS request is being made to host 'openaipublic.blob.core.windows.net'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


File already exists and is up to date


c:\Users\hp\anaconda3\Lib\site-packages\urllib3\connectionpool.py:1100: InsecureRequestWarning: Unverified HTTPS request is being made to host 'openaipublic.blob.core.windows.net'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


...

File already exists and is up to date


c:\Users\hp\anaconda3\Lib\site-packages\urllib3\connectionpool.py:1100: InsecureRequestWarning: Unverified HTTPS request is being made to host 'openaipublic.blob.core.windows.net'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


File already exists and is up to date


c:\Users\hp\anaconda3\Lib\site-packages\urllib3\connectionpool.py:1100: InsecureRequestWarning: Unverified HTTPS request is being made to host 'openaipublic.blob.core.windows.net'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


File already exists and is up to date


In [16]:
print("Settings:", settings)
print("Parameters dictionry keys :", params.keys())

Settings: {'n_vocab': 50257, 'n_ctx': 1024, 'n_embd': 768, 'n_head': 12, 'n_layer': 12}
Parameters dictionry keys : dict_keys(['blocks', 'b', 'g', 'wpe', 'wte'])


In [17]:
print(params['wte'])
print(params['wte'].shape)

[[-0.11010301 -0.03926672  0.03310751 ... -0.1363697   0.01506208
   0.04531523]
 [ 0.04034033 -0.04861503  0.04624869 ...  0.08605453  0.00253983
   0.04318958]
 [-0.12746179  0.04793796  0.18410145 ...  0.08991534 -0.12972379
  -0.08785918]
 ...
 [-0.04453601 -0.05483596  0.01225674 ...  0.10435229  0.09783269
  -0.06952604]
 [ 0.1860082   0.01665728  0.04611587 ... -0.09625227  0.07847701
  -0.02245961]
 [ 0.05135201 -0.02768905  0.0499369  ...  0.00704835  0.15519823
   0.12067825]]
(50257, 768)


In [18]:
#defining model configuration in a dictionary
model_config = {
    "gpt-2-small(124M)":{"emb_dim":768, "n_layers":12,"n_heads":12},
    "gpt-2-medium(355M)":{"emb_dim":768, "n_layers":24,"n_heads":16},
    "gpt-2-large(124M)":{"emb_dim":768, "n_layers":36,"n_heads":20},
    "gpt-2-xl(124M)":{"emb_dim":768, "n_layers":48,"n_heads":25}
}

In [19]:
#updating base configuration with specific model settings
model_name = "gpt-2-small(124M)"
NEW_CONFIG = GPT_CONFIG_124M.copy()
NEW_CONFIG.update(model_config[model_name])

In [20]:
NEW_CONFIG

{'vocab_size': 50257,
 'context_length': 256,
 'emb_dim': 768,
 'n_heads': 12,
 'n_layers': 12,
 'drop_rate': 0.1,
 'qkv_bias': False}

In [21]:
NEW_CONFIG.update({"context_length":1024, "qkv_bias":True})

In [22]:
NEW_CONFIG

{'vocab_size': 50257,
 'context_length': 1024,
 'emb_dim': 768,
 'n_heads': 12,
 'n_layers': 12,
 'drop_rate': 0.1,
 'qkv_bias': True}

In [23]:
myGpt_2 = GPTModel(NEW_CONFIG)

In [24]:
myGpt_2.eval()

GPTModel(
  (tok_emb): Embedding(50257, 768)
  (pos_emb): Embedding(1024, 768)
  (drop_emb): Dropout(p=0.1, inplace=False)
  (trf_blocks): Sequential(
    (0): TransformerBlock(
      (attn): MultiHeadAttention(
        (w_query): Linear(in_features=768, out_features=768, bias=True)
        (w_key): Linear(in_features=768, out_features=768, bias=True)
        (w_value): Linear(in_features=768, out_features=768, bias=True)
        (out_proj): Linear(in_features=768, out_features=768, bias=True)
        (dropout): Dropout(p=0.1, inplace=False)
      )
      (ff): FeedForward(
        (layers): Sequential(
          (0): Linear(in_features=768, out_features=3072, bias=True)
          (1): GELU()
          (2): Linear(in_features=3072, out_features=768, bias=True)
        )
      )
      (norm1): LayerNorm()
      (norm2): LayerNorm()
      (drop_shortcut): Dropout(p=0.1, inplace=False)
    )
    (1): TransformerBlock(
      (attn): MultiHeadAttention(
        (w_query): Linear(in_features

In [25]:
def assign(left, right):
    if left.shape != right.shape:
        raise ValueError(f"Shape mismatch: left = {left.shape}, right = {right.shape}")
    with torch.no_grad():
        left.copy_(torch.tensor(right))
    return left


In [26]:
import numpy as np

def load_weights_to_gpt(gpt,params):
    gpt.pos_emb.weight = assign(gpt.pos_emb.weight,params['wpe'] )
    gpt.tok_emb.weight = assign(gpt.tok_emb.weight, params['wte'])
    # loading weights to transformer blocks and it's each layer
    for b in range(len(params['blocks'])):
        #upadting key,query and value weights and bias
        q_w, k_w, v_w = np.split(
            (params["blocks"][b]["attn"]["c_attn"])["w"], 3, axis=-1)
        gpt.trf_blocks[b].attn.w_query.weight = assign(
            gpt.trf_blocks[b].attn.w_query.weight, q_w.T)
        gpt.trf_blocks[b].attn.w_key.weight = assign(
            gpt.trf_blocks[b].attn.w_key.weight, k_w.T)
        gpt.trf_blocks[b].attn.w_value.weight = assign(
            gpt.trf_blocks[b].attn.w_value.weight, v_w.T)
        
        q_b, k_b, v_b = np.split(
            (params["blocks"][b]["attn"]["c_attn"])["b"], 3, axis=-1)
        gpt.trf_blocks[b].attn.w_query.bias = assign(
            gpt.trf_blocks[b].attn.w_query.bias, q_b)
        gpt.trf_blocks[b].attn.w_key.bias = assign(
            gpt.trf_blocks[b].attn.w_key.bias, k_b)
        gpt.trf_blocks[b].attn.w_value.bias = assign(
            gpt.trf_blocks[b].attn.w_value.bias, v_b)
        
        #updating out projection weights and bias
        gpt.trf_blocks[b].attn.out_proj.weight = assign(
            gpt.trf_blocks[b].attn.out_proj.weight, 
            params["blocks"][b]["attn"]["c_proj"]["w"].T)
        gpt.trf_blocks[b].attn.out_proj.bias = assign(
            gpt.trf_blocks[b].attn.out_proj.bias, 
            params["blocks"][b]["attn"]["c_proj"]["b"])
        
        # updating weight of feed forward neural network 
        gpt.trf_blocks[b].ff.layers[0].weight = assign(
            gpt.trf_blocks[b].ff.layers[0].weight, 
            params["blocks"][b]["mlp"]["c_fc"]["w"].T)
        gpt.trf_blocks[b].ff.layers[0].bias = assign(
            gpt.trf_blocks[b].ff.layers[0].bias, 
            params["blocks"][b]["mlp"]["c_fc"]["b"])
        gpt.trf_blocks[b].ff.layers[2].weight = assign(
            gpt.trf_blocks[b].ff.layers[2].weight, 
            params["blocks"][b]["mlp"]["c_proj"]["w"].T)
        gpt.trf_blocks[b].ff.layers[2].bias = assign(
            gpt.trf_blocks[b].ff.layers[2].bias, 
            params["blocks"][b]["mlp"]["c_proj"]["b"])
        
        # updating weights for layer normalization scale and shift
        gpt.trf_blocks[b].norm1.scale = assign(
            gpt.trf_blocks[b].norm1.scale, 
            params["blocks"][b]["ln_1"]["g"])
        gpt.trf_blocks[b].norm1.shift = assign(
            gpt.trf_blocks[b].norm1.shift, 
            params["blocks"][b]["ln_1"]["b"])
        gpt.trf_blocks[b].norm2.scale = assign(
            gpt.trf_blocks[b].norm2.scale, 
            params["blocks"][b]["ln_2"]["g"])
        gpt.trf_blocks[b].norm2.shift = assign(
            gpt.trf_blocks[b].norm2.shift, 
            params["blocks"][b]["ln_2"]["b"])
    # End of transformer blocks
    gpt.final_norm.scale = assign(gpt.final_norm.scale, params['g'])
    gpt.final_norm.shift = assign(gpt.final_norm.shift, params['b'])
    gpt.out_head.weight = assign(gpt.out_head.weight, params['wte'])

In [27]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [28]:
load_weights_to_gpt(myGpt_2, params=params)

In [29]:
def text_to_token(text, tokenizer):
    encoded = tokenizer.encode(text, allowed_special={'<|endoftext|>'})
    encoded_tensor = torch.tensor(encoded).unsqueeze(0)
    return encoded_tensor
def token_to_text(ids, tokenizer):
    flat = ids.squeeze(0)
    return tokenizer.decode(flat.tolist())

In [30]:
import tiktoken
tokenizer = tiktoken.get_encoding("gpt2")

In [31]:
def generate(model,idx,max_new_token,context_size,temp=0.0,top_k=None,eos_id= None):
    for _ in range(max_new_token):
        idx_cond = idx[:,-context_size:]
        with torch.no_grad():
            logits = model(idx_cond)
        logits = logits[:,-1,:]
        # applying filter for top k
        if top_k is not None:
            top_logits,_  = torch.topk(logits,top_k)
            min_val = top_logits[:,-1]
            logits = torch.where(logits<min_val,torch.tensor(float("-inf")).to(device),logits)
        # Apply temprature scaling
        if temp>0.0:
            logits = logits/temp
            probas = torch.softmax(logits, dim=-1)
            idx_next = torch.multinomial(probas, num_samples=1)
        else:
            idx_next = torch.argmax(logits, dim=-1, keepdim=True)
        
        if idx_next==eos_id:
            break
        idx = torch.cat((idx,idx_next),dim=-1)

    return idx


In [32]:
torch.manual_seed(123)
token_ids   = generate(model=myGpt_2, idx=text_to_token("That day was special and a bit lucky for me",tokenizer).to(device),
                       max_new_token=50,context_size=NEW_CONFIG['context_length'],
                       top_k=50,temp=1.5)
print('Output Text: ', token_to_text(token_ids,tokenizer))

Output Text:  That day was special and a bit lucky for me," Icahn told Sporting News from his home in Austin, Texas. "It started off as a little set of circumstances that I couldn't live by, but actually played through it that way. Now I understand it now on how my mom helped


## Fine Tunning the model for classification task

In [33]:
import pandas as pd
df = pd.read_csv('SMSSpamCollection', sep='\t',header=None, names=['label','text'])

In [34]:
df

,label,text
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."
...,...,...
5567,spam,This is the 2nd time we have tried 2 contact u...
5568,ham,Will ü b going to esplanade fr home?
5569,ham,"Pity, * was in mood for that. So...any other s..."
5570,ham,The guy did some bitching but I acted like i'd...


In [35]:
print(df['label'].value_counts())

ham     4825
spam     747
Name: label, dtype: int64


In [38]:
def create_balanced_dataset(df):
    num_spam = df[df['label']=='spam'].shape[0]
    ham_subset = df[df['label']=='ham'].sample(num_spam,random_state = 123)
    balanced_df = pd.concat([ham_subset, df[df['label']=='spam']])
    return balanced_df

In [39]:
balanced_df = create_balanced_dataset(df)
balanced_df

,label,text
4307,ham,Awww dat is sweet! We can think of something t...
4138,ham,Just got to &lt;#&gt;
4831,ham,"The word ""Checkmate"" in chess comes from the P..."
4461,ham,This is wishing you a great day. Moji told me ...
5440,ham,Thank you. do you generally date the brothas?
...,...,...
5537,spam,Want explicit SEX in 30 secs? Ring 02073162414...
5540,spam,ASKED 3MOBILE IF 0870 CHATLINES INCLU IN FREE ...
5547,spam,Had your contract mobile 11 Mnths? Latest Moto...
5566,spam,REMINDER FROM O2: To get 2.50 pounds free call...


In [40]:
balanced_df['label'] = balanced_df['label'].map({'ham':0, 'spam':1})

In [41]:
balanced_df.head()

,label,text
4307,0,Awww dat is sweet! We can think of something t...
4138,0,Just got to &lt;#&gt;
4831,0,"The word ""Checkmate"" in chess comes from the P..."
4461,0,This is wishing you a great day. Moji told me ...
5440,0,Thank you. do you generally date the brothas?


In [45]:
def random_split(df, train_frac, val_frac):
    df = df.sample(frac=1,random_state=123).reset_index(drop=True)
    train_end = int(len(df)*train_frac)
    validation_end = train_end + int(len(df)*val_frac)
    #splitting the dataframe
    train_df  = df[:train_end]
    validation_df = df[train_end:validation_end]
    test_df = df[validation_end:]
    return train_df,validation_df,test_df

In [48]:
train_df,validation_df,test_df = random_split(balanced_df,0.7,0.1)

In [49]:
print(len(train_df))
print(len(validation_df))
print(len(test_df))

1045
149
300


In [50]:
train_df.to_csv('train.csv',index=None)
validation_df.to_csv('validation.csv', index=None)
test_df.to_csv('test.csv', index=None)

In [52]:
import torch 
from torch.utils.data import Dataset
class SpamdataSet(Dataset):
    def __init__(self,csvfile,tokenizer,max_length= None, pad_token_id = 50256):
        self.data = pd.read_csv(csvfile)
        #pretokenizing
        self.encoded_texts = [tokenizer.encode(text) for text in self.data['text']]
        if max_length is None:
            self.max_length = self._longest_encoded_length()
        else:
            self.max_length = max_length
            self.encoded_texts = [
                encoded_text[:max_length] for encoded_text in self.encoded_texts
            ]
        self.encoded_texts = [
            encoded_text + [pad_token_id]*(self.max_length - len(encoded_text))
            for encoded_text in self.encoded_texts
        ]
    
    def __getitem__(self, index):
        encoded = self.encoded_texts[index]
        label  = self.data.iloc[index]['label']
        return (torch.tensor(encoded,dtype=torch.long),
                torch.tensor(encoded,dtype=torch.long)
            )
    def __len__(self):
        return len(self.data)

    def _longest_encoded_length(self):
        max_length =0
        for encoded_t in self.encoded_texts:
            enc_len = len(encoded_t)
            if enc_len> max_length:
                max_length = enc_len
        return max_length


In [53]:
train_dataset = SpamdataSet(
    csvfile='train.csv',tokenizer=tokenizer
)

In [55]:
train_dataset.max_length

120

In [56]:
val_dataset = SpamdataSet(csvfile='train.csv', max_length=train_dataset.max_length,tokenizer=tokenizer)
test_dataset = SpamdataSet(csvfile='test.csv',max_length=train_dataset.max_length,tokenizer=tokenizer)

In [57]:
# creating dataloder from the dataset
import torch 
from torch.utils.data import DataLoader

num_workers = 0
batch_size =8
torch.manual_seed(123)

train_loader = DataLoader(
    dataset=train_dataset,
    batch_size=batch_size,
    num_workers=num_workers,
    drop_last=True
)
val_loader = DataLoader(
    dataset=val_dataset,
    batch_size=batch_size, 
    num_workers=num_workers,
    drop_last=False
)
test_loader = DataLoader(
    dataset=test_dataset,
    batch_size=batch_size,
    num_workers=num_workers,
    drop_last=False
)

In [59]:
len(train_loader)

130